In [1]:
import pandas as pd
from lxml import html
import requests

filename = "url_Scraping.csv"
df = pd.read_csv("data.csv")

df = df[0:10]

for i in range(0,len(df)):
    try:
        url_addr = f"https://{str(df.iloc[i,0])}"
        page = requests.get(f"{url_addr}")
        tree = html.fromstring(page.content)
        page_url = page.url
        title = tree.xpath('//html/head/title/text()')
        description = tree.xpath('//html/head/meta[@name="description"]/@content')
        ## REMOVE COMMENTS TO SAVE INTO url_Scrapping.csv
        
        #with open(filename, 'a') as f:
        #    f.write(f"{page_url.replace(',',';')},{title.replace(',',';')},{description.replace(',',';')} \n")
        #self.log(f"scrap added to {filename}")
        print("================")
        print(page_url)
        print(title)
        print(description)
        #print(f"{page_url.replace(',',';')},{title.replace(',',';')},{description.replace(',',';')} \n")
        print("================")
    except:
        print(f"Skipped url {i}, error scrapping")

https://droit-finances.commentcamarche.com/faq/15494-salaire-d-une-assistante-maternelle-ce-qu-il-faut-savoir
["Salaire d'une assistante maternelle : ce qu'il faut savoir"]
["La rémunération des assistantes maternelles (salaire horaire, indemnités d'entretien et de nourriture...) doit respecter certains montants minimum. Voici un point sur les droits de l'assistante maternelle et les obligations des parents en..."]
https://droit-finances.commentcamarche.com/forum/affich-5732576-ne-pas-declarer-ses-revenus-a-pole-emploi
['Ne pas déclarer ses revenus à pôle emploi [Résolu]']
['Meilleure réponse: Bonsoir, La déclaration mensuelle est obligatoire et doit comportée impérativement tous les revenus que vous avez pu touché pour que justement Pôle Emploi puisse calculé vos droits. Pensez à ceci : Votre employeur va vous déclaré...']
https://droit-finances.commentcamarche.com/faq/4843-aah-2019-montant-et-plafonds-de-ressources
['AAH 2019 : montant et plafonds de ressources']
["Le montant maximum

In [3]:
#Preprocessing URL data and creating url_Preprocess.csv
import pandas as pd

df = pd.read_csv("url_Scraping.csv", delimiter=",")
df = df.dropna(subset=['title', 'description']).drop(columns="Unnamed: 3")

for i in range(0,len(df)):
    # taking care of www., .com, http and https.. and tokenizing
    if df.iloc[i,0][4]=="s":
        df.iloc[i,0] = str(df.iloc[i,0][8:]).replace("www.","").replace(".comment",". comment").replace(".madame",". madame").replace(".com","").replace(".php","").replace(".fr","").replace(".html","").replace("."," ").replace("-"," ").replace("0","").replace("1","").replace("2","").replace("3","").replace("4","").replace("5","").replace("6","").replace("7","").replace("8","").replace("9","").strip().split("/")
        df.iloc[i,0] = str(" ".join(df.iloc[i,0]).strip().replace("     "," ").replace("  "," ").lower().replace(" artfig","")).split(" ")
    else:
        df.iloc[i,0] = str(df.iloc[i,0][7:]).replace("www.","").replace(".comment",". comment").replace(".madame",". madame").replace(".com","").replace(".php","").replace(".fr","").replace(".html","").replace("."," ").replace("-"," ").replace("0","").replace("1","").replace("2","").replace("3","").replace("4","").replace("5","").replace("6","").replace("7","").replace("8","").replace("9","").strip().split("/")
        df.iloc[i,0] = str(" ".join(df.iloc[i,0]).strip().replace("     "," ").replace("  "," ").lower().replace(" artfig","")).split(" ")

df.to_csv("url_Preprocess.csv")

In [4]:
import gensim
import pandas as pd
from gensim import corpora
import pickle

df = pd.read_csv("url_Preprocess.csv")
unallowed_words = ["lefigaro","linternaute","commentcamarche","journaldunet","lachainemeteo","linternaute"]

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3 and token not in unallowed_words:
            result.append(token)
    return result

processed_url = df['url_clean'].map(preprocess)
processed_url

dictionary = corpora.Dictionary(processed_url)
corpus = [dictionary.doc2bow(text) for text in processed_url]

pickle.dump(corpus, open('corpus.pkl', 'wb'))
#dictionary.save('dictionary.gensim')

NUM_TOPICS = 8
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
#ldamodel.save(f"model{NUM_TOPICS}.gensim")

In [5]:
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.058*"meteo" + 0.040*"fiche" + 0.040*"questionnaire" + 0.033*"ville" + 0.033*"weekend" + 0.029*"previsions" + 0.026*"aujourdhui" + 0.024*"deco" + 0.017*"coloriages" + 0.017*"hugolescargot"
Topic: 1 
Words: 0.052*"sante" + 0.047*"article" + 0.032*"conseils" + 0.019*"editorial" + 0.018*"cadremploi" + 0.016*"pour" + 0.015*"vous" + 0.011*"sont" + 0.009*"comment" + 0.007*"actualite"
Topic: 2 
Words: 0.044*"finances" + 0.043*"droit" + 0.024*"immobilier" + 0.015*"pour" + 0.015*"guide" + 0.014*"article" + 0.013*"meghan" + 0.013*"management" + 0.013*"markle" + 0.013*"prince"
Topic: 3 
Words: 0.124*"forum" + 0.121*"affich" + 0.028*"comment" + 0.014*"pour" + 0.014*"auto" + 0.011*"bricolage" + 0.010*"enfants" + 0.010*"plus" + 0.009*"faire" + 0.008*"avec"
Topic: 4 
Words: 0.092*"cuisine" + 0.055*"recette" + 0.053*"madame" + 0.045*"recettes" + 0.015*"gastronomie" + 0.015*"sport" + 0.010*"idees" + 0.009*"beaute" + 0.008*"foot" + 0.007*"pour"
Topic: 5 
Words: 0.084*"madame" + 0.043*"

In [6]:
topics = ldamodel.print_topics(num_words=1)
for topic in topics:
    print(topic)

(0, '0.058*"meteo"')
(1, '0.052*"sante"')
(2, '0.044*"finances"')
(3, '0.124*"forum"')
(4, '0.092*"cuisine"')
(5, '0.084*"madame"')
(6, '0.050*"etre"')
(7, '0.073*"sante"')


## Baseline Model: Analysing Titles with LDA

In [7]:
import gensim
import pandas as pd
from gensim import corpora
import pickle

df = pd.read_csv("url_Preprocess.csv")
unallowed_words = ["lefigaro","figaro","linternaute","commentcamarche","journaldunet","lachainemeteo","linternaute","internaute","madame"]

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 4 and token not in unallowed_words:
            result.append(token)
    return result

processed_url = df['title'].map(preprocess)
processed_url

dictionary = corpora.Dictionary(processed_url)
corpus = [dictionary.doc2bow(text) for text in processed_url]

NUM_TOPICS = 9
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
#ldamodel.save(f"model{NUM_TOPICS}.gensim")

In [15]:
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.071*"meteo" + 0.041*"météo" + 0.040*"jours" + 0.039*"gratuite" + 0.038*"chaîne" + 0.036*"prévisions" + 0.035*"résolu" + 0.009*"espagne" + 0.008*"académie" + 0.008*"résultat"
Topic: 1 
Words: 0.031*"conseils" + 0.021*"lettre" + 0.021*"fiches" + 0.019*"médicaux" + 0.018*"santé" + 0.016*"modèle" + 0.015*"tirage" + 0.014*"résultat" + 0.011*"villes" + 0.011*"ligne"
Topic: 2 
Words: 0.143*"recette" + 0.043*"facile" + 0.042*"cuisine" + 0.019*"meilleure" + 0.017*"faire" + 0.016*"france" + 0.009*"chocolat" + 0.008*"maison" + 0.008*"euros" + 0.008*"tarte"
Topic: 3 
Words: 0.111*"résolu" + 0.051*"santé" + 0.045*"médecine" + 0.039*"comment" + 0.016*"pourquoi" + 0.010*"symptômes" + 0.008*"nouveau" + 0.008*"bonne" + 0.008*"résumé" + 0.008*"cette"
Topic: 4 
Words: 0.020*"meghan" + 0.020*"définition" + 0.017*"markle" + 0.011*"dictionnaire" + 0.010*"facile" + 0.010*"générale" + 0.010*"simple" + 0.010*"votre" + 0.007*"fêtes" + 0.006*"match"
Topic: 5 
Words: 0.029*"gratuit" + 0.025*"co

In [8]:
topics = ldamodel.print_topics(num_words=1)
for topic in topics:
    print(topic)

(0, '0.029*"résultat"')
(1, '0.022*"résolu"')
(2, '0.019*"promo"')
(3, '0.097*"résolu"')
(4, '0.131*"recette"')
(5, '0.019*"cette"')
(6, '0.070*"meteo"')
(7, '0.066*"santé"')
(8, '0.027*"résolu"')


## Baseline Model: Analysing Descriptions with LDA

In [9]:
import gensim
import pandas as pd
from gensim import corpora
import pickle

df = pd.read_csv("url_Preprocess.csv")

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 4:
            result.append(token)
    return result

processed_url = df['description'].map(preprocess)
processed_url

dictionary = corpora.Dictionary(processed_url)
corpus = [dictionary.doc2bow(text) for text in processed_url]

#pickle.dump(corpus, open('corpus.pkl', 'wb'))
#dictionary.save('dictionary.gensim')

NUM_TOPICS = 9
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
#ldamodel.save(f"model{NUM_TOPICS}.gensim")

In [18]:
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.057*"réponse" + 0.055*"meilleure" + 0.019*"bonjour" + 0.014*"votre" + 0.011*"faire" + 0.006*"comme" + 0.006*"notre" + 0.006*"permet" + 0.005*"problème" + 0.005*"avoir"
Topic: 1 
Words: 0.017*"meteo" + 0.016*"altitude" + 0.016*"latitude" + 0.016*"longitude" + 0.008*"europe" + 0.008*"espagne" + 0.005*"située" + 0.004*"partie" + 0.004*"maladie" + 0.004*"symptômes"
Topic: 2 
Words: 0.026*"découvrez" + 0.013*"coloriage" + 0.012*"résultats" + 0.010*"retrouvez" + 0.008*"figaro" + 0.007*"classement" + 0.007*"millions" + 0.007*"sélection" + 0.007*"paris" + 0.006*"notre"
Topic: 3 
Words: 0.026*"recette" + 0.024*"verbe" + 0.018*"temps" + 0.011*"facile" + 0.010*"personnes" + 0.010*"ingrédients" + 0.010*"conjugaison" + 0.009*"sucre" + 0.009*"minutes" + 0.009*"avoir"
Topic: 4 
Words: 0.009*"après" + 0.006*"depuis" + 0.005*"images" + 0.005*"macron" + 0.005*"france" + 0.005*"vidéo" + 0.005*"cette" + 0.005*"monde" + 0.004*"lundi" + 0.004*"femme"
Topic: 5 
Words: 0.008*"voici" + 0.006

In [19]:
topics = ldamodel.print_topics(num_words=1)
for topic in topics:
    print(topic)

(0, '0.057*"réponse"')
(1, '0.017*"meteo"')
(2, '0.026*"découvrez"')
(3, '0.026*"recette"')
(4, '0.009*"après"')
(5, '0.008*"voici"')
(6, '0.028*"défaut"')
(7, '0.010*"santé"')
(8, '0.014*"voici"')


## Improving model: Combined URL/TITLES/DESCRIPTIONS

In [23]:
import gensim
import pandas as pd
from gensim import corpora
import pickle

df = pd.read_csv("url_Preprocess.csv")
unallowed_words = ["lefigaro","figaro","linternaute","commentcamarche","journaldunet","lachainemeteo","linternaute","internaute","madame"]

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 4 and token not in unallowed_words:
            result.append(token)
    return result

processed_url = df[['description', 'title', 'url_clean']].apply(lambda x: ' '.join(x).replace('[','').replace(']','').replace("'","").replace(".","").replace(",","").replace(";",""), axis=1).map(preprocess)
processed_url

dictionary = corpora.Dictionary(processed_url)
corpus = [dictionary.doc2bow(text) for text in processed_url]

#pickle.dump(corpus, open('corpus.pkl', 'wb'))
#dictionary.save('dictionary.gensim')

NUM_TOPICS = 9
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
#ldamodel.save(f"model{NUM_TOPICS}.gensim")

In [24]:
for idx, topic in ldamodel.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.045*"meteo" + 0.020*"france" + 0.015*"ville" + 0.012*"jours" + 0.011*"météo" + 0.011*"aujourdhui" + 0.010*"chaîne" + 0.010*"gratuite" + 0.010*"découvrez" + 0.009*"previsions"
Topic: 1 
Words: 0.040*"forum" + 0.040*"réponse" + 0.039*"meilleure" + 0.038*"affich" + 0.033*"résolu" + 0.019*"comment" + 0.019*"verbe" + 0.013*"bonjour" + 0.013*"conjugaison" + 0.008*"temps"
Topic: 2 
Words: 0.012*"celebrites" + 0.011*"video" + 0.009*"fiche" + 0.009*"défaut" + 0.008*"coloriage" + 0.008*"questionnaire" + 0.008*"description" + 0.008*"meghan" + 0.008*"photos" + 0.007*"horoscope"
Topic: 3 
Words: 0.006*"voyage" + 0.005*"idées" + 0.005*"voici" + 0.004*"guide" + 0.004*"couleur" + 0.004*"vacances" + 0.004*"calendrier" + 0.004*"video" + 0.003*"nouveau" + 0.003*"cette"
Topic: 4 
Words: 0.028*"conseils" + 0.013*"article" + 0.010*"comment" + 0.009*"cadremploi" + 0.008*"hallyday" + 0.008*"votre" + 0.008*"editorial" + 0.007*"johnny" + 0.007*"dictionnaire" + 0.007*"astuces"
Topic: 5 
Words:

In [25]:
topics = ldamodel.print_topics(num_words=1)
for topic in topics:
    print(topic)

(0, '0.045*"meteo"')
(1, '0.040*"forum"')
(2, '0.012*"celebrites"')
(3, '0.006*"voyage"')
(4, '0.028*"conseils"')
(5, '0.040*"sante"')
(6, '0.058*"recette"')
(7, '0.019*"droit"')
(8, '0.018*"article"')
